In [11]:
import re
import os
import pandas as pd
import ipywidgets as ipw
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

directorio_credenciales = 'credentials_module.json'
def login():
    gauth = GoogleAuth()
    gauth.LoadCredentialsFile(directorio_credenciales)
    
    if gauth.access_token_expired:
        gauth.Refresh()
        gauth.SaveCredentialsFile(directorio_credenciales)
    else:
        gauth.Authorize()
        
    return GoogleDrive(gauth)
drive = login()

data_id2020I1  = pd.read_excel(r'Expedientes_2020.xlsx')
data_id2020I2  = pd.read_excel(r'ID_2020.xlsx')
data_id2021I1  = pd.read_excel(r'Expedientes_2021.xlsx')
data_id2021I2  = pd.read_excel(r'ID_2021.xlsx')
data_id2022I1  = pd.read_excel(r'Expedientes_2022.xlsx')
data_id2022I2  = pd.read_excel(r'ID_2022.xlsx')
data_usuarios  = pd.read_excel(r'UsuariosSearchIT.xlsx')

def acceder(value):
    if value == 'OsinergminSC':
        VBox4.layout.display = 'None'
        HBox5.layout.visibility  = 'visible'
        for i in range(data_usuarios.shape[0]):
            if Text2.value == '':
                idexe = '1-0NmNO5L38WH2C_SWSLCCoP1f4nc-tfv'
            elif Text2.value in data_usuarios.iat[i,0]:
                idexe = data_usuarios.iat[i,1]
                break
            else:
                idexe = '1-0NmNO5L38WH2C_SWSLCCoP1f4nc-tfv'
        url    = "https://docs.google.com/spreadsheets/d/" + idexe + "/edit#gid=0"
        tag    = "_blank"
        font   = '4'
        HTML8.value = f"<font size={font}><a href={url} target={tag}>Documento Excel</a>"
        Text1.value = '¡ Hola ' + Text2.value + ' !'

def searchwords(text, keys):
    separ = re.findall('\(.*?\)', keys)
    words = [[] for i in range(len(separ))]
    j     = -1
    for i in separ:
        j = j + 1
        if len(i.replace('(', '').replace(')', '').split(',')) == 1:
            words[j] = i.replace('(', '').replace(')', '')
        else:
            for k in i.replace('(', '').replace(')', '').replace(' ', '').split(','):
                words[j].append(k)
    tandf = [''] * len(words)
    j     = -1
    for i in words:
        j = j + 1
        if isinstance(words[j], list):
            torf = [''] * len(words[j])
            n    = -1
            for k in words[j]:
                n = n + 1
                if k[0] == '-':
                    k = k.replace('-', '')
                    if k not in text:
                        torf[n] = 1
                    else:
                        torf[n] = 0
                else:
                    if k in text:
                        torf[n] = 1
                    else:
                        torf[n] = 0
            if 1 in torf:
                tandf[j] = 1
            else:
                tandf[j] = 0
        else:
            if i[0] == '-':
                i = i.replace('-', '')
                if i not in text:
                    tandf[j] = 1
                else:
                    tandf[j] = 0
            else:
                if i in text:
                    tandf[j] = 1
                else:
                    tandf[j] = 0
    if 0 in tandf:
        rpt = 0
    else:
        rpt = 1
    return rpt

def showdoc(value):
    naexp = Text1.value
    idexp = '1As_Rcolpc8s3YzTOolXisjJY9HQvC22H'
    if Dropdown1.value == '2020':
        for i in range(data_id2020I2.shape[0]):
            if naexp in data_id2020I2.iat[i,0]:
                idexp = data_id2020I2.iat[i,1]
                break
    elif Dropdown1.value == '2021':
        for i in range(data_id2021I2.shape[0]):
            if naexp in data_id2021I2.iat[i,0]:
                idexp = data_id2021I2.iat[i,1]
                break
    elif Dropdown1.value == '2022':
        for i in range(data_id2022I2.shape[0]):
            if naexp in data_id2022I2.iat[i,0]:
                idexp = data_id2022I2.iat[i,1]
                break
    width  = "650px"
    height = "600px"
    border = "0"
    urls   = "https://drive.google.com/file/d/" + idexp + "/preview"
    HTML9.value = f"<iframe src={urls} width={width} height={height} frameborder={border}>"
    Text1.value = str(i)
    
def findexp(value):
    IntProgress1.value = 0
    IntProgress1.bar_style = 'success'
    areas = [''] * 6
    areas[0] = Textarea1.value; areas[1] = Textarea2.value; areas[2] = Textarea3.value;
    areas[3] = Textarea4.value; areas[4] = Textarea5.value; areas[5] = Textarea6.value;
    extract = []
    if Dropdown1.value == '2020':
        IntProgress1.max = data_id2020I2.shape[0]
        for i in range(IntProgress1.max):
            expediente = data_id2020I1.iat[i,0]; concesionaria = data_id2020I1.iat[i,1]; materia = data_id2020I1.iat[i,2]
            sumilla = data_id2020I1.iat[i,3]; vistos = data_id2020I1.iat[i,4]; analisis = data_id2020I1.iat[i,5]; resolucion = data_id2020I1.iat[i,6]
            infor = [concesionaria, materia, sumilla, vistos, analisis, resolucion]
            k     = -1
            for area in areas:
                k = k + 1
                if area != '':
                    if searchwords(infor[k], area) == 0:
                        break
                if k == (len(infor) - 1):
                    extract.append([expediente, concesionaria, materia, sumilla])
            IntProgress1.value = i + 1
            if (i + 1) == IntProgress1.max:
                IntProgress1.bar_style = 'danger'
    elif Dropdown1.value == '2021':
        IntProgress1.max = data_id2021I2.shape[0]
        for i in range(IntProgress1.max):
            expediente = data_id2021I1.iat[i,0]; concesionaria = data_id2021I1.iat[i,1]; materia = data_id2021I1.iat[i,2]
            sumilla = data_id2021I1.iat[i,3]; vistos = data_id2021I1.iat[i,4]; analisis = data_id2021I1.iat[i,5]; resolucion = data_id2021I1.iat[i,6]
            infor = [concesionaria, materia, sumilla, vistos, analisis, resolucion]
            k     = -1
            for area in areas:
                k = k + 1
                if area != '':
                    if searchwords(infor[k], area) == 0:
                        break
                if k == (len(infor) - 1):
                    extract.append([expediente, concesionaria, materia, sumilla])
            IntProgress1.value = i + 1
            if (i + 1) == IntProgress1.max:
                IntProgress1.bar_style = 'danger'
    elif Dropdown1.value == '2022':
        IntProgress1.max = data_id2022I2.shape[0]
        for i in range(IntProgress1.max):
            expediente = data_id2022I1.iat[i,0]; concesionaria = data_id2022I1.iat[i,1]; materia = data_id2022I1.iat[i,2]
            sumilla = data_id2022I1.iat[i,3]; vistos = data_id2022I1.iat[i,4]; analisis = data_id2022I1.iat[i,5]; resolucion = data_id2022I1.iat[i,6]
            infor = [concesionaria, materia, sumilla, vistos, analisis, resolucion]
            k     = -1
            for area in areas:
                k = k + 1
                if area != '':
                    if searchwords(infor[k], area) == 0:
                        break
                if k == (len(infor) - 1):
                    extract.append([expediente, concesionaria, materia, sumilla])
            IntProgress1.value = i + 1
            if (i + 1) == IntProgress1.max:
                IntProgress1.bar_style = 'danger'
    if extract == []:
        extract.append(['NO', 'SE', 'ENCONTRARON', 'RESULTADOS'])
    df = pd.DataFrame(extract)
    df.columns = ['Expediente', 'Concesionaria', 'Materia', 'Sumilla']
    df.to_excel("OsiExpedientes.xlsx", index = None)
    for i in range(data_usuarios.shape[0]):
        if Text2.value == '':
            idexe = '1-0NmNO5L38WH2C_SWSLCCoP1f4nc-tfv'
        elif Text2.value in data_usuarios.iat[i,0]:
            idexe = data_usuarios.iat[i,1]
            break
        else:
            idexe = '1-0NmNO5L38WH2C_SWSLCCoP1f4nc-tfv'
    gfile = drive.CreateFile({'id': idexe})
    gfile.SetContentFile(r'OsiExpedientes.xlsx')
    gfile.Upload()

HTML1 = ipw.HTML(value = "<font size='5'><b>Concesionaria</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML2 = ipw.HTML(value = "<font size='5'><b>Materia</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML3 = ipw.HTML(value = "<font size='5'><b>Sumilla</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML4 = ipw.HTML(value = "<font size='5'><b>Antecedentes</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML5 = ipw.HTML(value = "<font size='5'><b>Análisis</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML6 = ipw.HTML(value = "<font size='5'><b>Resolución</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
HTML7 = ipw.HTML(value = "<font size='5'><b>Expediente</b></font>", layout = ipw.Layout(padding = "10px 20px 0px 0px"))
url    = "https://docs.google.com/spreadsheets/d/1-0NmNO5L38WH2C_SWSLCCoP1f4nc-tfv/edit#gid=0"
tag    = "_blank"
font   = '4'
HTML8  = ipw.HTML(value = f"<font size={font}><a href={url} target={tag}>Documento Excel</a>")
width  = "650px"
height = "580px"
border = "0"
urls   = "https://drive.google.com/file/d/1za4EHG-tQ5YnsEXUaYQWvUydtzZIL0yR/preview"
HTML9  = ipw.HTML(value = f"<iframe src={urls} width={width} height={height} frameborder={border}>", layout = ipw.Layout(margin = "0px 0px 0px 35px"))
HTML10 = ipw.HTML(value = "<font size='20'><b>SearchIT</b></font>", layout = ipw.Layout(padding = "60px 0px 0px 210px"))
HTML11 = ipw.HTML(value = "<marquee style='font-size: 2; width: 100%' scrollamount='10'><b>Desarrollado por: Junior Aguilar</b></marquee>", layout = ipw.Layout(padding = "5px 0px 0px 0px"))
HTML12 = ipw.HTML(value = "<font size='20'><b>SearchIT</b></font>", layout = ipw.Layout(padding = "0px 0px 15px 60px"))
Textarea1 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Textarea2 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Textarea3 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Textarea4 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Textarea5 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Textarea6 = ipw.Textarea(layout = ipw.Layout(width = '435px', padding = "2px 10px 0px 0px"))
Text1 = ipw.Text(layout = ipw.Layout(width = '200px', heigth = '100px', padding = "10px 0px 0px 33px"))
Text2 = ipw.Text(value = '', placeholder ='Ingrese su usuario', description = 'Usuario:', layout = ipw.Layout(width = '300px', height = '30px', padding = '0px', margin = '0px', border = '0px'))
Dropdown1 = ipw.Dropdown(options = ['2020', '2021', '2022'], layout = ipw.Layout(width = '96px', heigth = '100px', padding = "10px 0px 0px 10px"))
Button1 = ipw.Button(description = 'Mostrar', layout = ipw.Layout(width = '150px', heigth = '120px', margin = "12px 0px 0px 10px"))
Button1.on_click(showdoc)
Button2 = ipw.Button(description = 'Buscar expedientes relacionados !', layout = ipw.Layout(width = '427px', heigth = '200px', margin = "0px 0px 0px 43px"))
Button2.on_click(findexp)
IntProgress1 = ipw.IntProgress(value = 0, min = 0, bar_style = 'success', orientation='horizontal', layout = ipw.Layout(width = '609px', heigth = '100px', margin = "7px 0px 0px 4px"))
VBox1 = ipw.VBox([HTML1, HTML2, HTML3, HTML4, HTML5, HTML6])
VBox2 = ipw.VBox([Textarea1, Textarea2, Textarea3, Textarea4, Textarea5, Textarea6])
HBox1 = ipw.HBox([VBox1, VBox2])
HBox2 = ipw.HBox([HTML7, Text1, Button1, Dropdown1])
HBox3 = ipw.HBox([IntProgress1], layout = ipw.Layout(margin = "5px 0px 0px 0px"))
HBox4 = ipw.HBox([HTML8, Button2], layout = ipw.Layout(margin = "10px 0px 0px 0px"))
VBox3 = ipw.VBox([HBox1, HBox2, HBox3, HBox4, HTML10, HTML11])
HBox5 = ipw.HBox([VBox3, HTML9])
Password1 = ipw.Password(value = '', placeholder ='Ingrese la contraseña de ingreso', description = 'Contraseña:', layout = ipw.Layout(width = '300px', height = '30px', padding = '0px', margin = '0px', border = '0px'))
f_acceder = ipw.interactive(acceder, value = Password1)
VBox4 = ipw.VBox([HTML12, Text2, Password1], layout = ipw.Layout(padding = '0px', margin = '200px 0px 0px 500px', border = '0px'))
display(VBox4)
display(HBox5)
HBox5.layout.visibility = "hidden"